# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db, hrzn_load
from asteroid_element import load_ast_elt, load_data_impl, convert_data
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [3]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [4]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [5]:
fname_csv = hrzn_txt2csv(fname_txt)

In [6]:
fname_csv

'/ssd1/tmp/mysql/jpl/010_sun.csv'

In [7]:
df = hrzn_csv2df(fname_csv)

In [8]:
pd.set_option('display.max_rows', 10)

In [9]:
df

,BodyTypeCD,BodyNumber,BodyName,IntegrationSource,JD,CalendarDateTime,delta_T,qx,qy,qz,vx,vy,vz
0,S,10,Sun,DE431mx,2440400.5,1969-06-28,39.699541,0.004503,0.000810,-0.000061,-3.517495e-07,0.000006,-1.438024e-08
1,S,10,Sun,DE431mx,2440401.5,1969-06-29,39.702104,0.004502,0.000815,-0.000061,-3.587527e-07,0.000006,-1.432102e-08
2,S,10,Sun,DE431mx,2440402.5,1969-06-30,39.704668,0.004502,0.000821,-0.000061,-3.657021e-07,0.000006,-1.426221e-08
3,S,10,Sun,DE431mx,2440403.5,1969-07-01,39.707232,0.004501,0.000827,-0.000061,-3.725998e-07,0.000006,-1.420343e-08
4,S,10,Sun,DE431mx,2440404.5,1969-07-02,39.709795,0.004501,0.000832,-0.000061,-3.794481e-07,0.000006,-1.414425e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37196,S,10,Sun,DE431mx,2477596.5,2071-04-30,69.185505,-0.000708,-0.002510,-0.000002,4.158106e-06,-0.000004,-8.460845e-08
37197,S,10,Sun,DE431mx,2477597.5,2071-05-01,69.185494,-0.000704,-0.002514,-0.000002,4.164001e-06,-0.000004,-8.473888e-08
37198,S,10,Sun,DE431mx,2477598.5,2071-05-02,69.185482,-0.000700,-0.002518,-0.000002,4.169895e-06,-0.000004,-8.487000e-08
37199,S,10,Sun,DE431mx,2477599.5,2071-05-03,69.185470,-0.000696,-0.002522,-0.000002,4.175791e-06,-0.000004,-8.500189e-08


## Testing - Populate Database from CSV

In [10]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [11]:
engine = create_engine(connection_str)

In [12]:
engine.table_names()

['AsteroidElement_Numbered',
 'AsteroidElement_Unnumbered',
 'HorizonsBody',
 'HorizonsImport',
 'HorizonsTime',
 'HorizonsVectors',
 'LargeBody',
 'SmallBody']

In [13]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [14]:
# df = load_ast_elt()

In [15]:
df_in = load_data_impl()

In [16]:
df_in

,Num,Name,Epoch,a,e,i,w,Node,M,H,G,Ref
Num,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,10.59407,73.59769,80.30553,77.372097,3.40,0.12,JPL 47
2,2,Pallas,59000.0,2.773841,0.229972,34.83293,310.20239,173.02474,144.975675,4.20,0.11,JPL 37
3,3,Juno,59000.0,2.668285,0.256936,12.99104,248.06619,169.85148,125.435355,5.33,0.32,JPL 112
4,4,Vesta,58600.0,2.361418,0.088721,7.14177,150.72854,103.81080,95.861938,3.00,0.32,JPL 35
5,5,Astraea,59000.0,2.574037,0.190913,5.36743,358.64842,141.57103,17.846343,6.90,0.15,JPL 114
...,...,...,...,...,...,...,...,...,...,...,...,...
1413590,1413590,6013 P-L,37203.0,2.280861,0.185919,5.98442,195.44280,194.22990,342.057565,17.14,0.15,JPL 5
1413591,1413591,6331 P-L,59000.0,2.334910,0.282919,8.08228,5.21622,355.24824,260.113042,18.50,0.15,JPL 8
1413592,1413592,6344 P-L,59000.0,2.817151,0.662446,4.67928,234.89485,182.98283,240.473601,20.40,0.15,JPL 17


In [17]:
df = convert_data(df_in[0:100])

In [18]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref
Num,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114
...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,Aegle,59000.0,3.048360,0.141178,0.278942,5.611058,3.638345,3.115040,7.70,0.15,JPL 97
97,97,Klotho,59000.0,2.669021,0.257298,0.205558,2.785849,4.689231,0.723346,7.80,0.15,JPL 97
98,98,Ianthe,59000.0,2.686497,0.187233,0.271867,6.176797,2.768649,3.757108,8.90,0.15,JPL 78


In [20]:
from asteroid_element import ast_data_add_calc_elements

In [21]:
df = ast_data_add_calc_elements(df)

100%|██████████| 100/100 [00:00<00:00, 121398.09it/s]

Making big simulation with all 100 asteroids...
Computing orbital elements...
Copying additional orbital elements to DataFrame...


In [22]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47,1.501306,1683.145658,0.003733,4.036516,4.187424,2.686118,-361.745861
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37,2.742191,1687.410960,0.003724,4.681022,4.892910,2.150719,-679.537622
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112,2.535135,1592.013779,0.003947,3.200114,3.545989,1.010854,-554.707814
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940426
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114,0.461775,1508.414438,0.004165,2.758769,2.909067,2.447291,-74.776891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,Aegle,59000.0,3.048360,0.141178,0.278942,5.611058,3.638345,3.115040,7.70,0.15,JPL 97,3.121407,1944.007406,0.003232,-0.201927,-0.195560,-3.316967,-963.788304
97,97,Klotho,59000.0,2.669021,0.257298,0.205558,2.785849,4.689231,0.723346,7.80,0.15,JPL 97,1.156078,1592.671892,0.003945,1.915241,2.347973,1.191895,-183.354893
98,98,Ianthe,59000.0,2.686497,0.187233,0.271867,6.176797,2.768649,3.757108,8.90,0.15,JPL 78,-2.706485,1608.340384,0.003907,0.136183,-0.044224,2.662261,646.613524


In [ ]:
import rebound
from tqdm import tqdm as tqdm_console

In [ ]:
df = ast_data_add_calc_elements(df)

In [ ]:
ast_elt = df.copy()

In [ ]:
# Number of asteroids
N: int = len(ast_elt)

# Initialize empty arrays for computed orbital elements
f = np.zeros(N)
P = np.zeros(N)
mean_motion = np.zeros(N)
long = np.zeros(N)
theta = np.zeros(N)
pomega = np.zeros(N)
T_peri = np.zeros(N)

# Base Rebound simulation with just the Sun
# We are NOT integrating this simulation, only using it to convert orbital elements
# Therefore we don't need the planets, or the initial configuration of the sun; just its mass.
sim = rebound.Simulation()
sim.units = ('day', 'AU', 'Msun')
sim.add(m=1.0)
sim.N_active = 1

# All the available asteroid numbers; wrap as a tqdm iterator for progress bar
# nums = ast_elt.index
nums = ast_elt.Num.values
iters = tqdm_console(nums)

# Make a gigantic simulation with all these asteroids
print(f'Making big simulation with all {N} asteroids...')
for num in iters:
    # Unpack the orbital elements
    a = ast_elt.a[num]
    e = ast_elt.e[num]
    inc = ast_elt.inc[num]
    Omega = ast_elt.Omega[num]
    omega = ast_elt.omega[num]
    M = ast_elt.M[num]
    # Set the primary to the sun (NOT the solar system barycenter!)
    primary = sim.particles[0]
    # Add the asteroid to the simulation as a massless test particle.  Just want the elements!
    sim.add(m=0.0, a=a, e=e, inc=inc, Omega=Omega, omega=omega, M=M, primary=primary)

# Calculate orbital elements for all particles; must specify primary = Sun!!!
print(f'Computing orbital elements...')
orbits = sim.calculate_orbits(primary=sim.particles[0])

# Iterate over all the asteroids in the simulation
print(f'Copying additional orbital elements to DataFrame...')
iters = list(enumerate(nums))
for i, num in tqdm_console(iters):
    # Look up the orbit of asteroid i
    orb = orbits[i]
    # Unpack the additional (calculated) orbital elements
    f[i] = orb.f
    P[i] = orb.P
    mean_motion[i] = orb.n
    long[i] = orb.l
    theta[i] = orb.theta
    pomega[i] = orb.pomega
    T_peri[i] = orb.T

# Save computed orbital elements to the DataFrame
ast_elt['f'] = f
ast_elt['P'] = P
ast_elt['n'] = mean_motion
ast_elt['long'] = long
ast_elt['theta'] = theta
ast_elt['pomega'] = pomega
ast_elt['T_peri'] = T_peri


In [ ]:
len(orbits)

In [ ]:
i

In [ ]:
orbits[0].a

In [ ]:
epoch_unq = df_in.Epoch.unique()
epoch_unq

In [ ]:
len(epoch_unq)

In [ ]:
df

In [ ]:
import rebound

In [ ]:
sim = rebound.Simulation()
sim.units = ('day', 'AU', 'Msun')

In [ ]:
sim.add(m=1.0)

In [ ]:
sim.particles[0]

In [ ]:
sim.add(m=0.0, a=2.77, e=0.23, inc=0.60, Omega=3.02, omega=5.41, M=2.53)

In [ ]:
p = sim.particles[1]
p

In [ ]:
p.f

In [ ]:
df = load_ast_elt()

In [ ]:
df = ast_data_add_calc_elements(df)

In [ ]:
df = convert_data(df_in)

In [ ]:
M = df.M.values[0:10]
e = df.e.values[0:10]
E0 = None

In [ ]:
anomaly_M2E_impl(M=M, e=e)

In [ ]:
M

In [ ]:
E = anomaly_M2E_impl(M, e)

In [ ]:
M

In [ ]:
E

In [ ]:
f = E - e * np.sin(E) - M
np.max(np.abs(f))

In [ ]:
# Put M in the interval [-pi, pi]
tau = 2.0 * np.pi
M += np.pi
M %= tau
M -= np.pi

In [ ]:
M

In [ ]:
# Use the initial guess E0 if provided; otherwise use M
E = E0 if E0 is not None else M

In [ ]:
E

In [ ]:
# Maximum number of iterations for Newton-Raphson
max_iter: int = 50

# Tolerance for maximum error
err_tol: np.float64 = 2.0**-50

In [ ]:
# The current function value f(E)
f = E - e * np.sin(E) - M
# Is the max error below the tolerance? If so, quit early
max_err = np.max(np.abs(f))

In [ ]:
f

In [ ]:
max_err

In [ ]:
# The derivative f'(E)
fp = 1.0 - e * np.cos(E)

In [ ]:
fp

In [ ]:
E -= f / fp

In [ ]:
E

In [ ]:
# The current function value f(E)
f = E - e * np.sin(E) - M
# Is the max error below the tolerance? If so, quit early
max_err = np.max(np.abs(f))
# The derivative f'(E)
fp = 1.0 - e * np.cos(E)
# Update E using Newton's method
E -= f / fp

In [ ]:
f

In [ ]:
fp

In [ ]:
max_err